<a href="https://colab.research.google.com/github/sanazMj/DSGAN/blob/master/Train_baseline_transferlearning_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50, DenseNet121, VGG16
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
from skimage.transform import resize
from io import BytesIO
from PIL import Image
import glob
import copy
import random

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
Project_path =  "/content/drive/MyDrive/Alzheimer_research/"
data_path = Project_path + '/Sanaz_Code/OASIS_data/New/'

In [3]:
img_width, img_height = 224, 224
fold_nums = 5
batch_num = 30
num_epochs = 50
num_repeats = 1
early_stopping_patience = 5
model_cats = ['Baseline', 'Transfer_learning', 'Fine_tuning']
model_kinds = [VGG16, ResNet50, DenseNet121]
kfold = KFold(n_splits=fold_nums, shuffle=True)

# Load OASIS data

In [ ]:
# data_path = Project_path + '/Sanaz_Code/OASIS_data/'
# Alzheimers_files = []
# NonAlzheimers_files = []

# counter = 0
# path, dirs, files = next(os.walk(data_path + '/Alzheimers/'))
# for d in dirs:
#   path, dirs, files = next(os.walk(data_path + '/Alzheimers/' + d))
#   for f in files:
#     counter+=1
#     file_select = data_path + 'Alzheimers/' + d +'/'+ f
#     print(counter)
#     os.rename(src = file_select, dst = data_path + '/New/'+'/Alzheimers/' + d + '_' + f )


# counter = 0
# path, dirs, files = next(os.walk(data_path + '/NonAlzheimer/'))
# for d in dirs:
#   path, dirs, files = next(os.walk(data_path + '/NonAlzheimer/' + d))
#   for f in files:
#     counter+=1
#     file_select = data_path + 'NonAlzheimer/' + d +'/'+ f
#     print(counter)
#     os.rename(src = file_select, dst = data_path + '/New/'+'/NonAlzheimer/' + d + '_' + f )

In [5]:
data_path = Project_path + '/Sanaz_Code/OASIS_data/New/'
Alzheimers_files = []
NonAlzheimers_files = []

path, dirs, files = next(os.walk(data_path +'/Alzheimers/'))
for f in files:
  Alzheimers_files += files

path, dirs, files = next(os.walk(data_path +'/NonAlzheimer/'))
for f in files: 
  NonAlzheimers_files += files

In [41]:
def get_data(num_folds, len_AD_class = 3200):
    

  train_len =  int(0.64 * len_AD_class)
  val_len =  int(0.16 * len_AD_class)
  test_len = int(0.20 * len_AD_class)

  y_AD = []
  X_AD = []
  y_NonAD = []
  X_NonAD = []

  for filename in os.listdir(Project_path + '/Sanaz_Code/OASIS_data/New/Alzheimers/'):

      temp = Image.open(Project_path + '/Sanaz_Code/OASIS_data/New/Alzheimers/' + filename)
      temp = temp.resize((img_width, img_height),Image.ANTIALIAS)
      temp = np.asarray(temp)
      RGB_X = np.stack((temp,)*3, axis = -1)
      # print(RGB_X.shape)

      X_AD.append(RGB_X)

      if len(X_AD) % 200 == 0:
        print(len(X_AD), " of the 3200 have been uploaded")
  for filename in os.listdir(Project_path + '/Sanaz_Code/OASIS_data/New/NonAlzheimer/'):

      temp = Image.open(Project_path + '/Sanaz_Code/OASIS_data/New/NonAlzheimer/' + filename)
      temp = temp.resize((img_width, img_height),Image.ANTIALIAS)
      temp = np.asarray(temp)
      RGB_X = np.stack((temp,)*3, axis = -1)
      # print(RGB_X.shape)

      X_NonAD.append(RGB_X)

      if len(X_NonAD) % 200 == 0:
        print(len(X_NonAD), " of the 3200 have been uploaded")
  
  X_test = np.concatenate((np.array(X_AD[:test_len]), np.array(X_NonAD[:test_len])))
  y_test = np.concatenate((np.ones(test_len), np.zeros(test_len)))
  
  X_non_test_AD =  X_AD[test_len:]
  X_non_test_nonAD =  X_NonAD[test_len:]

  folds_X_train =[]
  folds_y_train = []
  
  folds_X_val =[]
  folds_y_val = []

  for i in range(num_folds):
    random.shuffle(X_non_test_AD)
    random.shuffle(X_non_test_nonAD)

    X_train = np.concatenate((np.array(X_non_test_AD[:train_len]), 
                              np.array(X_non_test_nonAD[:train_len])))
    X_val = np.concatenate((np.array(X_non_test_AD[train_len:train_len+val_len]),
                            np.array(X_non_test_nonAD[train_len:train_len+val_len])))
    
    folds_X_train.append(X_train)
    folds_y_train.append(np.concatenate((np.ones(train_len), np.zeros(train_len))))

    folds_X_val.append(X_val)
    folds_y_val.append(np.concatenate((np.ones(val_len), np.zeros(val_len))))
  
  return X_test, y_test, folds_X_train, folds_y_train, folds_X_val, folds_y_val

In [42]:
X_test, y_test, folds_X_train, folds_y_train, folds_X_val, folds_y_val = get_data(3, len_AD_class = 3200)
print(X_test.shape, y_test.shape, folds_X_train[0].shape, folds_y_train[0].shape,
      folds_X_val[0].shape, folds_y_val[0].shape)

200  of the 3200 have been uploaded
400  of the 3200 have been uploaded
600  of the 3200 have been uploaded
800  of the 3200 have been uploaded
1000  of the 3200 have been uploaded
1200  of the 3200 have been uploaded
1400  of the 3200 have been uploaded
1600  of the 3200 have been uploaded
1800  of the 3200 have been uploaded
2000  of the 3200 have been uploaded
2200  of the 3200 have been uploaded
2400  of the 3200 have been uploaded
2600  of the 3200 have been uploaded
2800  of the 3200 have been uploaded
3000  of the 3200 have been uploaded
3200  of the 3200 have been uploaded
200  of the 3200 have been uploaded
400  of the 3200 have been uploaded
600  of the 3200 have been uploaded
800  of the 3200 have been uploaded
1000  of the 3200 have been uploaded
1200  of the 3200 have been uploaded
1400  of the 3200 have been uploaded
1600  of the 3200 have been uploaded
1800  of the 3200 have been uploaded
2000  of the 3200 have been uploaded
2200  of the 3200 have been uploaded
2400  of 

# Load ADNI data

In [ ]:
# dataset_path = Project_path + '/ADNI_data/'
# X = np.load(dataset_path + '/Data.npy')
# y = np.load(dataset_path + '/Labels.npy')
# print(X.shape, y.shape)

In [43]:
params_dict = {'Transfer': {'initial_weight':'imagenet', 'trainable_layer':False},
               'Fine_tuning': {'initial_weight':'imagenet', 'trainable_layer':False},
               'Baseline': {'initial_weight':None, 'trainable_layer':True} }

# Evaluation functions

In [44]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Model functions

In [52]:
def create_model(model_kind, trainable_layers, add_type, dropouts, model_cat='Baseline', input_size=224):
    set_trainable = False
    
    Model = Sequential()
    Model.add(model_kind(include_top=False, pooling='avg', weights = params_dict[model_cat]['initial_weight'], input_shape=(input_size, input_size, 3)))

    for layer in Model.layers:
      if model_cat == 'Fine_tuning':
         if layer.name in trainable_layers:
          layer.trainable = False
         else:
          layer.trainable = True
      else:
        layer.trainable = params_dict[model_cat]['trainable_layer']
      
    Model.add(Flatten())
    if add_type == 0:
      Model.add(Dense(256, activation='relu'))
      Model.add(Dropout(dropouts[0]))
    elif add_type == 1:
      Model.add(Dense(256, activation='relu'))
      Model.add(Dropout(dropouts[0]))
      Model.add(Dense(128, activation='relu'))
      Model.add(Dropout(dropouts[1]))

    # VGG.add(BatchNormalization())

    # VGG.add(BatchNormalization())
    # VGG.add(Dense(2048, activation='relu'))
    # VGG.add(BatchNormalization())
    Model.add(Dense(1, activation='sigmoid'))

    print(Model.summary())
    # Check the trainable status of the individual layers
    for layer in Model.layers:
        print(layer, layer.trainable)

    opt = RMSprop(learning_rate = 0.001)
    # opt = Adam(learning_rate=0.01)
    Model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    # Model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
    return Model


## ADNI Model_training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)

for repeat in range(1, 6):
    for model_kind in model_kinds:
      for model_cat in model_cats:
            if model_caat == 'Fine_tuning':
              trainable_layers = []
            else:
              trainable_layers = []

            fold_no = 1
            Vacc_per_fold = []
            Vloss_per_fold = []

            # perform cross validation
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

            for train, val in kfold.split(X_train, y_train):

                # csv_logger = CSVLogger(dataset_path + 'training_'+ str(model_name) + str(transfer_learning)+'r_'+str(repeat)+'_'+ str(fold_no)+'.log')

                model = create_model(model_name, trainable_layers, model_cat=model_cat):
                print(f'Training for fold {fold_no} ...', X_train[train].shape[0], X_train[val].shape[0])

                history = model.fit(X_train[train], y_train[train],
                                  batch_size=batch_num,
                                  epochs=num_epochs,
                                  verbose=1, validation_data=(X_train[val], y_train[val]),
                                  callbacks=[early_stopping])

                # Generate generalization metrics
                scores = model.evaluate(X_test, y_test, verbose=0)
                print(scores)
                print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1] * 100}%')

                y_pred = model.predict(X_test, batch_size=64, verbose=1)
                y_pred_bool = np.argmax(y_pred, axis=1)
                print(classification_report(y_test, y_pred_bool))

                y_pred_reshaped = y_pred.reshape(y_pred.shape[0])
                predictor = np.array([round(num) for num in y_pred_reshaped])

                # print(prediction.shape, validation_labels.shape)
                Confus = confusion_matrix(y_test, predictor)
                print(Confus)


                Vacc_per_fold.append(scores[1] * 100)
                Vloss_per_fold.append(scores[0])

                # Increase fold number
                fold_no = fold_no + 1

            print(Vacc_per_fold, Vloss_per_fold)

## OASIS model training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)

fold_nums = 5
num_repeats = 5
num_epochs = 30
add_types = [0]
dropouts = [[0.5], [0.2]]
model_kinds = [VGG16]
model_cats =  ['Baseline']
for repeat in range(1, num_repeats+1):
    for model_name in model_kinds:
      for model_cat in model_cats:
        for add_type in add_types:
          for dropout in dropouts:
            
            
            Vacc_per_fold = []
            Vloss_per_fold = []

            if model_cat == 'Fine_tuning':
              trainable_layers = []
            else:
              trainable_layers = []

            for i in range(fold_nums):
                print('repeat', repeat, 'fold', i)

                model = create_model(model_name, trainable_layers, add_type, dropout, model_cat=model_cat)

                history = model.fit(folds_X_train[i], folds_y_train[i], batch_size=batch_num,
                                  epochs=num_epochs,
                                  verbose=1,
                                  validation_data=(folds_X_val[i], folds_y_val[i]), 
                                   callbacks=[early_stopping])     

                # Generate generalization metrics
                scores = model.evaluate(X_test, verbose=0)
                print(scores)
                print(f'Score for fold {i}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1] * 100}%')

                y_pred = model.predict(X_test, batch_size=64, verbose=1)
                y_pred_bool = np.argmax(y_pred, axis=1)
                print(classification_report(y_test, y_pred_bool))

                y_pred_reshaped = y_pred.reshape(y_pred.shape[0])
                predictor = np.array([round(num) for num in y_pred_reshaped])

                # print(prediction.shape, validation_labels.shape)
                Confus = confusion_matrix(y_test, predictor)
                print(Confus)


                Vacc_per_fold.append(scores[1] * 100)
                Vloss_per_fold.append(scores[0])


repeat 1 fold 0
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 14,846,273
Trainable params: 14,846,273
Non-trainable params: 0
_________________________________________________________________
None
<tensorflow.python.keras.engine.functional.Functional object at 0x7f558f1b5190> True
<t

# Appendix

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)

nb_train_samples = 5120
nb_validation_samples = 1280
fold_no = 0
fold_nums = 1
num_repeats = 1
num_epochs = 2
model_kinds = [VGG16]
model_cats =  ['Baseline']
for repeat in range(1, num_repeats+1):
    for model_name in model_kinds:
      for model_cat in model_cats:
            Vacc_per_fold = []
            Vloss_per_fold = []
            fold_no = 0

            if model_cat == 'Fine_tuning':
              trainable_layers = []
            else:
              trainable_layers = []


            for i in range(fold_nums):
                train_data_dir = data_path + '/data' + str(i) + '/train/'
                validation_data_dir = data_path + '/data' + str(i) + '/validation'

                train_labels = np.array(
                        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
                
                validation_labels = np.array(
                    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
                
                datagen = ImageDataGenerator(rescale=1. / 255)

                train_generator = datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_num,
                    class_mode='binary',
                    shuffle=False)
                val_generator = datagen.flow_from_directory(
                    validation_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_num,
                    class_mode='binary',
                    shuffle=False)
                print('Data created')

                model = create_model(model_name, trainable_layers, model_cat=model_cat)
                print(f'Training for fold {fold_no} ...')
                history = model.fit(train_generator, epochs=num_epochs,steps_per_epoch=10,
                                              validation_data=val_generator, validation_steps=10, 
                                              verbose=1, callbacks=[early_stopping])     

                # Generate generalization metrics
                scores = model.evaluate(val_generator, verbose=0)
                print(scores)
                print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1] * 100}%')

                y_pred = model.predict(val_generator, batch_size=64, verbose=1)
                y_pred_bool = np.argmax(y_pred, axis=1)
                print(classification_report(validation_labels, y_pred_bool))

                y_pred_reshaped = y_pred.reshape(y_pred.shape[0])
                predictor = np.array([round(num) for num in y_pred_reshaped])

                # print(prediction.shape, validation_labels.shape)
                Confus = confusion_matrix(validation_labels, predictor)
                print(Confus)


                Vacc_per_fold.append(scores[1] * 100)
                Vloss_per_fold.append(scores[0])

                # Increase fold number
                fold_no =+1 

In [ ]:



def get_data_files(AD_files, non_AD_files, kfold):


  fold_train_data = []
  fold_val_data = []

  train_len =  int(0.64 * len(AD_files))
  val_len =  int(0.16 * len(AD_files))
  test_len = int(0.20 * len(AD_files))

  print(len(AD_files), len(non_AD_files))

  # AD_test = np.random.choice(AD_files, size=test_len, replace=False)
  # NonAD_test = np.random.choice(non_AD_files, size=test_len, replace=False)
  random.shuffle(AD_files)
  random.shuffle(non_AD_files)

  AD_test = AD_files[:test_len]
  NonAD_test = non_AD_files[:test_len]

  test_set = [AD_test, NonAD_test]
  
  AD_files_after_test_indexes = AD_files[test_len:]
  NonAD_files_after_test_indexes =  non_AD_files[test_len:]

  print(len(AD_files_after_test_indexes), len(NonAD_files_after_test_indexes))


  for i in range(kfold):

    random.shuffle(AD_files_after_test_indexes)
    random.shuffle(NonAD_files_after_test_indexes)

    AD_train = AD_files_after_test_indexes[:train_len]
    NonAD_train = NonAD_files_after_test_indexes[:train_len]

    AD_val = AD_files_after_test_indexes[train_len: ]
    NonAD_val = NonAD_files_after_test_indexes[train_len:]
  
    fold_train_data.append([AD_train, NonAD_train])
    fold_val_data.append([AD_val, NonAD_val])

    print('AD datasets:', len(AD_train), len(AD_val), len(AD_test))
    print('Non_AD datasets:', len(NonAD_train), len(NonAD_val), len(NonAD_test))

  return fold_train_data, fold_val_data , test_set

get_data_files(Alzheimers_files, NonAlzheimers_files, 2)

In [ ]:

# def get_data(batch_num, len_AD_class = 3200)
    

#     train_len =  int(0.64 * len_AD_class//batch_num)
#     val_len =  int(0.16 * len_AD_class//batch_num)
#     test_len = int(0.20 * len_AD_class//batch_num)

#     train_data_AD = image_dataset_from_directory(
#     Project_path + '/Sanaz_Code/OASIS_data/New/Alzheimers/', labels=None, shuffle=True)

#     train_data_NonAD = image_dataset_from_directory(
#     Project_path + '/Sanaz_Code/OASIS_data/New/NonAlzheimer/', labels=None, shuffle=True)
    
#     for x, y in train_data:
#       print(x.shape)
#       ds = a.concatenate(b)

#       break